### Deep Learning with Keras on Amazon SageMaker

Last update: January 21st, 2020

Amazon SageMaker is a modular, fully managed Machine Learning service that lets you easily build, train and deploy models at any scale.

In this notebook, we'll use Keras (with the TensorFlow backend) to build a simple Convolutional Neural Network (CNN). We'll then train it to classify the Fashion-MNIST image data set. Fashion-MNIST is a Zalando dataset consisting of a training set of 60,000 examples and a validation set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes: it's a drop-in replacement for MNIST.

Resources
  * Amazon SageMaker documentation [ https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html ]
  * SageMaker SDK 
    * Code [ https://github.com/aws/sagemaker-python-sdk ] 
    * Documentation [ https://sagemaker.readthedocs.io/ ]
  * Fashion-MNIST [ https://github.com/zalandoresearch/fashion-mnist ] 
  * Keras documentation [ https://keras.io/ ]
  * Numpy documentation [ https://docs.scipy.org/doc/numpy/index.html ]
  
### https://gitlab.com/juliensimon/aim410
### Twitter: @julsimon

## Import the latest SageMaker SDK

In [1]:
%%sh
pip install --upgrade pip
pip -q install sagemaker smdebug smdebug-rulesconfig==0.1.2 awscli boto3 keras --upgrade 

Requirement already up-to-date: pip in /home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages (20.0.2)


In [4]:
! pip uninstall tensorflow -y

Found existing installation: tensorflow 2.0.1
Uninstalling tensorflow-2.0.1:
  Successfully uninstalled tensorflow-2.0.1


In [5]:
! pip install tensorflow==2.1.0

  Using cached tensorflow-2.0.1-cp36-cp36m-manylinux2010_x86_64.whl (86.3 MB)


In [31]:
from IPython.core.display import HTML
HTML("<script>Jupyter.notebook.kernel.restart()</script>")

In [2]:
import sagemaker

print(sagemaker.__version__)
sess = sagemaker.Session()
role = sagemaker.get_execution_role()

1.50.14.post0


## Download the Fashion-MNIST dataset

In [ ]:
from IPython.display import Image
Image("fashion-mnist-sprite.png")

First, we need to download the data set from the Internet. Fortunately, Keras provides a simple way to do this. The data set is already split (training and validation), with separate Numpy arrays for samples and labels. 

We create a local directory, and save the training and validation data sets separately.

In [3]:
import os
import keras
import numpy as np
from keras.datasets import fashion_mnist

(x_train, y_train), (x_val, y_val) = fashion_mnist.load_data()

os.makedirs("./data", exist_ok = True)

np.savez('./data/training', image=x_train, label=y_train)
np.savez('./data/validation', image=x_val, label=y_val)

Using TensorFlow backend.


In [6]:
%%sh
ls -l data

total 53668
-rw-rw-r-- 1 ec2-user ec2-user 47100466 Feb 25 01:06 training.npz
-rw-rw-r-- 1 ec2-user ec2-user  7850466 Feb 25 01:06 validation.npz


## Take a look at our Keras code

In [ ]:
%%sh
pygmentize train/code/mnist_keras_tf20_eager.py

The main steps are:
  * receive and parse command line arguments: five hyper parameters, and four environment variables (we'll get back to these in a moment)
  * load the data sets
  * make sure data sets have the right shape for TensorFlow (channels last)
  * normalize data sets, i.e. tranform [0-255] pixel values to [0-1] values
  * one-hot encode category labels (not familiar with this? More info: [ https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/ ])
  * Build a Sequential model in Keras: two convolution block with max pooling, followed by a fully connected layer with dropout, and a final classification layer. Don't worry if this sounds like gibberish, it's not our focus today
  * Train the model, leveraging multiple GPUs if they're available.
  * Print statistics
  * Save the model in TensorFlow serving format
  

## Train outside of SageMaker (just like on your laptop)

Before we start training on SageMaker, let's run this code locally and make sure that it trains fine. We just need to set the four environment variables that it expects, and run the script with Python

In [7]:
import tensorflow as tf
print(tf.__version__)

2.0.1


In [8]:
!pip uninstall tensorflow-serving

In [10]:
!pip install tensorflow-serving-api==2.1.0

     |████████████████████████████████| 421.8 MB 19 kB/s s eta 0:00:01   |                                | 471 kB 34.0 MB/s eta 0:00:13     |██████████████████              | 238.3 MB 63.1 MB/s eta 0:00:03
  Using cached scipy-1.4.1-cp36-cp36m-manylinux1_x86_64.whl (26.1 MB)
     |████████████████████████████████| 3.8 MB 44.2 MB/s eta 0:00:01
     |████████████████████████████████| 448 kB 47.5 MB/s eta 0:00:01
  Using cached six-1.14.0-py2.py3-none-any.whl (10 kB)
ERROR: tensorboard 2.1.0 has requirement grpcio>=1.24.3, but you'll have grpcio 1.10.1 which is incompatible.
  Attempting uninstall: scipy
    Found existing installation: scipy 1.3.1
    Uninstalling scipy-1.3.1:
      Successfully uninstalled scipy-1.3.1
  Attempting uninstall: six
    Found existing installation: six 1.11.0
    Uninstalling six-1.11.0:
      Successfully uninstalled six-1.11.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.0.2
    Uninstalling tensorboard-2.0.2:
      

In [11]:
# Number  GPUs on this machine
%env SM_NUM_GPUS=0
# Where to save the model
%env SM_MODEL_DIR=/tmp/model
# Where the training data is
%env SM_CHANNEL_TRAINING=data
# Where the validation data is
%env SM_CHANNEL_VALIDATION=data

!/bin/rm -rf $SM_MODEL_DIR
!python train/code/mnist_keras_tf20_eager.py --epochs 1

env: SM_NUM_GPUS=0
env: SM_MODEL_DIR=/tmp/model
env: SM_CHANNEL_TRAINING=data
env: SM_CHANNEL_VALIDATION=data
2020-02-25 01:10:37.503197: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-10.0/lib64:/usr/local/cuda-10.0/extras/CUPTI/lib64:/usr/local/cuda-10.0/lib:/usr/local/cuda-10.0/efa/lib:/opt/amazon/efa/lib:/opt/amazon/efa/lib64:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/:/usr/lib64/openmpi/lib/:/usr/local/lib:/usr/lib:/usr/local/mpi/lib:/lib/::/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/tensorflow
2020-02-25 01:10:37.503345: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.

Why are these environment variables important anyway? Well, they will be automatically passed to our script by SageMaker, so that we know where the data sets are, where to save the model, and how many GPUs we have. So, if you write your local code this way, **there won't be anything to change** to run it on SageMaker.

This feature is called '**script mode**', it's the recommended way to work with built-in frameworks on SageMaker.

In [12]:
!tree /tmp/model

/tmp/model
└── 1
    ├── assets
    ├── saved_model.pb
    └── variables
        ├── variables.data-00000-of-00001
        └── variables.index

3 directories, 3 files


## Train on the notebook instance (aka 'local mode')

Our code runs fine. Now, let's try to run it inside the built-in TensorFlow environment provided by SageMaker. For fast experimentation, let's use local mode.

In [16]:
from sagemaker.tensorflow import TensorFlow

tf_estimator = TensorFlow(entry_point='mnist_keras_tf20_eager.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='local',
                          framework_version='2.0.0', 
                          py_version='py3',
                          script_mode=True,
                          hyperparameters={'epochs': 1}
                         )

Now, let's define the local location of the training and validation data sets

In [17]:
local_training_input_path   = 'file://data/training.npz'
local_validation_input_path = 'file://data/validation.npz'

Let's train!

In [18]:
tf_estimator.fit({'training': local_training_input_path, 'validation': local_validation_input_path})

Creating tmpvei5sq_e_algo-1-kskcv_1 ... 
Attaching to tmpvei5sq_e_algo-1-kskcv_12mdone
algo-1-kskcv_1  | 2020-02-25 01:14:26,250 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
algo-1-kskcv_1  | 2020-02-25 01:14:26,258 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kskcv_1  | 2020-02-25 01:14:26,449 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kskcv_1  | 2020-02-25 01:14:26,468 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kskcv_1  | 2020-02-25 01:14:26,485 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-kskcv_1  | 2020-02-25 01:14:26,497 sagemaker-containers INFO     Invoking user script
algo-1-kskcv_1  | 
algo-1-kskcv_1  | Training Env:
algo-1-kskcv_1  | 
algo-1-kskcv_1  | {
algo-1-kskcv_1  |     "additional_framework_parameters": {},
algo-1-kskcv_1  |     "channel_input_dirs": {
algo-1-kskcv_1  

OK, our job runs fine locally. Let's now run the same job on a managed instance.

## Upload the data set to S3

SageMaker training instances expect data sets to be stored in Amazon S3, so let's upload them there. We could use boto3 to do this, but the SageMaker SDK includes a simple function: [Session.upload_data()](https://sagemaker.readthedocs.io/en/stable/session.html).



*Note: for high-performance workloads, Amazon EFS and Amazon FSx for Lustre are now also supported. More info [here](https://aws.amazon.com/blogs/machine-learning/speed-up-training-on-amazon-sagemaker-using-amazon-efs-or-amazon-fsx-for-lustre-file-systems/).

In [19]:
prefix = 'keras-fashion-mnist'

# Upload the training data set to 'keras-fashion-mnist/training'
training_input_path   = sess.upload_data('data/training.npz', key_prefix=prefix+'/training')

# Upload the validation data set to 'keras-fashion-mnist/validation'
validation_input_path = sess.upload_data('data/validation.npz', key_prefix=prefix+'/validation')

print(training_input_path)
print(validation_input_path)

s3://sagemaker-us-east-1-731833107751/keras-fashion-mnist/training/training.npz
s3://sagemaker-us-east-1-731833107751/keras-fashion-mnist/validation/validation.npz


We're done with our data set. Of course, in real life, much more work would be needed for data cleaning and preparation!

## Train with Managed Spot Training, and enable debugging with Amazon SageMaker Debugger

EC2 Spot Instances have long been a great cost optimization feature, and spot training is now available on SageMaker.
This blog [post](https://aws.amazon.com/blogs/aws/managed-spot-training-save-up-to-90-on-your-amazon-sagemaker-training-jobs/) has more info.

We're also using Amazon SageMaker Debugger to check for unwanted training conditions. **ZERO KERAS CODE NEEDED!**

In [20]:
# Configure a managed training job for 'mnist_keras_tf.py', 
# using a single c5.2xlarge spot instance running TensorFlow 1.15 in script mode

from sagemaker.tensorflow import TensorFlow
from sagemaker.debugger import Rule, rule_configs

tf_estimator = TensorFlow(entry_point='train/code/mnist_keras_tf20_eager.py', 
                          role=role,
                          train_instance_count=1, 
                          #train_instance_type='ml.p3.2xlarge',
                          train_instance_type='ml.c5.xlarge',
                          framework_version='2.0.0', 
                          py_version='py3',
                          script_mode=True,
                          train_use_spot_instances=True,        # Use spot instance
                          #train_use_spot_instances=False,        # Use spot instance
                          train_max_run=1200,                    # Max training time
                          train_max_wait=3600,                  # Max training time + spot waiting time
                          rules = [Rule.sagemaker(rule_configs.loss_not_decreasing()),
                                   Rule.sagemaker(rule_configs.overfit())]
                         )

Let's train!

In [21]:
# Train on the training and validation data sets stored in S3

tf_estimator.fit({'training': training_input_path, 'validation': validation_input_path})

2020-02-25 01:19:50 Starting - Starting the training job...
2020-02-25 01:20:20 Starting - Launching requested ML instances
********* Debugger Rule Status *********
*
*  LossNotDecreasing: InProgress        
*            Overfit: InProgress        
*
****************************************
......
2020-02-25 01:21:04 Starting - Preparing the instances for training...
2020-02-25 01:21:50 Downloading - Downloading input data...
2020-02-25 01:22:19 Training - Training image download completed. Training in progress...2020-02-25 01:22:23,320 sagemaker-containers INFO     Imported framework sagemaker_tensorflow_container.training
2020-02-25 01:22:23,326 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-02-25 01:22:31,049 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-02-25 01:22:31,061 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
2020-02-25 01:22:31,073 sagemaker-containers INFO     No GPUs det

This will take about 10 minutes. Please take a look at the training log. The first few lines show SageMaker preparing the managed instance. While the job is training, you can also look at metrics in the AWS console for SageMaker, and at the training log in the the AWS console for CloudWatch Logs.

Let's check the status of the debug rules we configured.

In [22]:
job_name = tf_estimator.latest_training_job.name
client = tf_estimator.sagemaker_session.sagemaker_client

description = client.describe_training_job(TrainingJobName=job_name)

In [23]:
import pprint 
for status in description['DebugRuleEvaluationStatuses']:
    status.pop('LastModifiedTime')
    status.pop('RuleEvaluationJobArn')
    pprint.pprint(status)

{'RuleConfigurationName': 'LossNotDecreasing',
 'RuleEvaluationStatus': 'Error',
 'StatusDetails': 'InternalServerError: We encountered an internal error. '
                  'Please try again.'}
{'RuleConfigurationName': 'Overfit',
 'RuleEvaluationStatus': 'Error',
 'StatusDetails': 'InternalServerError: We encountered an internal error. '
                  'Please try again.'}


Let's also look at tensor information saved in S3.

In [24]:
s3_output_path = description["DebugHookConfig"]["S3OutputPath"] + job_name + '/' + 'debug-output/'

print(s3_output_path)

s3://sagemaker-us-east-1-731833107751/tensorflow-training-2020-02-25-01-19-50-077/debug-output/


In [25]:
%%sh -s "$s3_output_path"

aws s3 ls --recursive $1

2020-02-25 01:28:34          0 tensorflow-training-2020-02-25-01-19-50-077/debug-output/training_job_end.ts


In [26]:
import smdebug
from smdebug.trials import create_trial

trial = create_trial(s3_output_path)
trial

[2020-02-25 01:34:09.270 ip-172-16-123-31:19718 INFO s3_trial.py:42] Loading trial  at path s3://sagemaker-us-east-1-731833107751/tensorflow-training-2020-02-25-01-19-50-077/debug-output/


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-26-23d035960cac>", line 4, in <module>
    trial = create_trial(s3_output_path)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/smdebug/trials/utils.py", line 18, in create_trial
    return S3Trial(name=name, bucket_name=bucket_name, prefix_name=prefix_name, **kwargs)
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/smdebug/trials/s3_trial.py", line 47, in __init__
    self._load_collections()
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/smdebug/trials/trial.py", line 168, in _load_collections
    _wait_for_collection_files(1)  # wait for the first collection file
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packa

MissingCollectionFiles: Training job has ended. All the collection files could not be loaded

In [27]:
trial.tensor_names()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-27-0dae1b2bce17>", line 1, in <module>
    trial.tensor_names()
NameError: name 'trial' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_l

NameError: name 'trial' is not defined

In [28]:
loss_values = trial.tensor('loss').values()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-28-0b3305e59201>", line 1, in <module>
    loss_values = trial.tensor('loss').values()
NameError: name 'trial' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinner

NameError: name 'trial' is not defined

In [29]:
loss_values

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 2963, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-29-67e725e72358>", line 1, in <module>
    loss_values
NameError: name 'loss_values' is not defined

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 1863, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'NameError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/ultratb.py", line 1095, in get_records
    return _fixed_getinnerframes(etb, number_of_line

NameError: name 'loss_values' is not defined

## Deploy to a real-time endpoint accelerated by Amazon Elastic Inference

Once the job is complete, the trained model is saved in S3, and is now ready to be deployed.

Elastic Inference is a feature that lets you attach fractional GPU acceleration to any EC2 instance. It's also available on SageMaker.

This blog [post](https://aws.amazon.com/blogs/aws/amazon-elastic-inference-gpu-powered-deep-learning-inference-acceleration/) has more info.

In [33]:
import time

# Deploy the model to an endpoint backed by a single c5.large instance,
# accelerated by a medium-size elastic inference accelerator

tf_endpoint_name = 'keras-tf-fmnist-'+time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

#tf_predictor = tf_estimator.deploy(initial_instance_count=1,
#                                   instance_type='ml.p2.xlarge', # $1.361/hour in eu-west-1
#                                   endpoint_name=tf_endpoint_name)      

tf_predictor = tf_estimator.deploy(initial_instance_count=1, # Performance comparable to p2.xlarge:
                         instance_type='ml.c5.large',        # $0.134/hour in eu-west-1
                         #accelerator_type='ml.eia1.medium',  # + $0.140/hour in eu-west-1
                         endpoint_name=tf_endpoint_name)     # = 80% discount :)

-----------------!

In [34]:
! pip list

Package                            Version      
---------------------------------- -------------
absl-py                            0.8.1        
alabaster                          0.7.10       
anaconda-client                    1.6.14       
anaconda-project                   0.8.2        
asn1crypto                         0.24.0       
astor                              0.8.0        
astroid                            1.6.3        
astropy                            3.0.2        
attrs                              18.1.0       
Automat                            0.3.0        
autovizwidget                      0.13.1       
awscli                             1.18.6       
Babel                              2.5.3        
backcall                           0.1.0        
backports.shutil-get-terminal-size 1.0.0        
bcrypt                             3.1.7        
beautifulsoup4                     4.6.0        
bitarray                           0.8.1        
bkcharts            

In [ ]:
from sagemaker.tensorflow.serving import Model

model = Model(model_data='s3://sagemaker-us-east-1-731833107751/keras-fashion-mnist/model.tar.gz', role=role)

predictor = model.deploy(initial_instance_count=1, instance_type='ml.c5.large')

## Automatic Model Tuning

Automatic model tuning is a great feature that helps you find automatically the best hyper parameters for your training job.

This blog [post](https://aws.amazon.com/blogs/machine-learning/amazon-sagemaker-automatic-model-tuning-now-supports-random-search-and-hyperparameter-scaling/) has more info.

First, let's define parameter ranges

In [ ]:
# Define parameter ranges :

from sagemaker.tuner import IntegerParameter, ContinuousParameter

hyperparameter_ranges = {
    'epochs' :       IntegerParameter(5, 20),
    'learning-rate': ContinuousParameter(0.001, 0.1, scaling_type='ReverseLogarithmic'), 
    'batch-size':    IntegerParameter(32, 1024),
    'filters':       IntegerParameter(4, 64),
    'dense-layer':   IntegerParameter(32, 1024),
    'dropout':       ContinuousParameter(0.2, 0.8)
}

The next step is to define the metric we're optimizing for, in this case we want to maximize the validation accuracy. We also grab other metrics from the training log.

In [ ]:
objective_metric_name = 'validation_accuracy'

objective_type = 'Maximize'

metric_definitions = [
    {'Name': 'training_loss',        'Regex': 'loss: ([0-9\\.]+)'},
    {'Name': 'training_accuracy',    'Regex': 'acc: ([0-9\\.]+)'},
    {'Name': 'validation_loss',      'Regex': 'val_loss: ([0-9\\.]+)'},
    {'Name': 'validation_accuracy',  'Regex': 'val_acc: ([0-9\\.]+)'},
    {'Name': 'training_precision',   'Regex': 'precision: ([0-9\\.]+)'},
    {'Name': 'training_recall',      'Regex': 'recall: ([0-9\\.]+)'},
    {'Name': 'training_f1_score',    'Regex': 'f1_score: ([0-9\\.]+)'},
    {'Name': 'validation_precision', 'Regex': 'val_precision: ([0-9\\.]+)'},
    {'Name': 'validation_recall',    'Regex': 'val_recall: ([0-9\\.]+)'},
    {'Name': 'validation_f1_score',  'Regex': 'val_f1_score: ([0-9\\.]+)'}
]

Then, it's time to put everything together, and configure the tuning job. Same estimator as above, without the debugging job.

In [ ]:
tf_estimator = TensorFlow(entry_point='mnist_keras_tf.py', 
                          role=role,
                          train_instance_count=1, 
                          train_instance_type='ml.p3.2xlarge',
                          framework_version='1.15', 
                          py_version='py3',
                          script_mode=True,
                          train_use_spot_instances=True,        # Use spot instance
                          train_max_run=600,                    # Max training time
                          train_max_wait=3600                   # Max training time + spot waiting time
                         )

In [ ]:
from sagemaker.tuner import HyperparameterTuner

# Configure a training job using the Tensorflow estimator, the parameter ranges and the metric defined above.
# Let's run ten individual jobs, two by two.

tuner = HyperparameterTuner(tf_estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            metric_definitions,
                            max_jobs=20,
                            max_parallel_jobs=2,
                            objective_type=objective_type)

Finally, let's launch the tuning job, just like a normal estimator. We definitely want to use spot training here!

In [ ]:
# Launch the tuning job, passing the location of the data sets in S3.

tuner.fit({'training': training_input_path, 'validation': validation_input_path})

While the job is running, you can view it in the AWS console for SageMaker: individual jobs (and their logs), best training job so far, etc.

Of course, you can also inspect the job programatically using [boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html) : *decribe_hyper_parameter_training_job()*, etc.

## Inspect jobs with Amazon SageMaker Experiments

Model tuning automatically creates a new experiment, and pushes information for each job. 

**ZERO KERAS CODE NEEDED!**

In [ ]:
from sagemaker.analytics import HyperparameterTuningJobAnalytics

exp = HyperparameterTuningJobAnalytics(
    sagemaker_session=sess, 
    hyperparameter_tuning_job_name=tuner.latest_tuning_job.name
)

df = exp.dataframe()
df

Pandas is the Swiss army knife for columnar data. Let's just look at the top job.

In [ ]:
best_job = df.sort_values('FinalObjectiveValue', ascending=0)[:1]
best_job

In [ ]:
best_job_name = best_job['TrainingJobName'].to_string(index=False).strip()
best_job_name

In [ ]:
import boto3
sm = boto3.client('sagemaker')

In [ ]:
best_job = sm.describe_training_job(TrainingJobName=best_job_name)

best_model_artefact = best_job['ModelArtifacts']['S3ModelArtifacts']
best_model_container = best_job['AlgorithmSpecification']['TrainingImage']

print(best_job_name)
print(best_model_artefact)
print(best_model_container)

## Deploy the best model, enabling data capture with Amazon SageMaker Model Monitor

This is where we want to save captured data.

In [ ]:
# Don't forget to update the bucket! It must be in the same region as SageMaker
s3_capture_path = 's3://jsimon-capture-euwest1/' + best_job_name + '/'

print(s3_capture_path)

By default, we will capture 100% of model inputs and outputs. Of course, this is configurable.

And you guessed it... **ZERO KERAS CODE NEEDED!**

In [ ]:
from sagemaker.model_monitor import DataCaptureConfig

cap = DataCaptureConfig(
    enable_capture=True,
    destination_s3_uri=s3_capture_path
)

In [ ]:
endpoint_name = best_job_name + '-ep'

best_model_predictor = tuner.deploy(
    initial_instance_count=1, 
    instance_type='ml.m4.xlarge', 
    endpoint_name=endpoint_name,
    data_capture_config=cap)

## Predict with best model

In [ ]:
%matplotlib inline
import random
import matplotlib.pyplot as plt

num_samples = 10
indices = random.sample(range(x_val.shape[0] - 1), num_samples)
images = x_val[indices]/255
labels = y_val[indices]

for i in range(num_samples):
    plt.subplot(1,num_samples,i+1)
    plt.imshow(images[i].reshape(28, 28), cmap='gray')
    plt.title(labels[i])
    plt.axis('off')
    
prediction = best_model_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
prediction = np.array(prediction)
predicted_labels = prediction.argmax(axis=1)
print('Predicted labels are: {}'.format(predicted_labels))

Now let's predict the validation dataset 250 samples at a time, storing labels and predicted labels as we go.

In [ ]:
%%time
num_samples = 250
all_labels=[]
all_predicted_labels=[]

import sys

for i in range(0,x_val.shape[0] - 1,num_samples):
    sys.stdout.write(str(i)+' ')
    indices = range(i,i+num_samples)
    images = x_val[indices]/255
    labels = y_val[indices]
    prediction = best_model_predictor.predict(images.reshape(num_samples, 28, 28, 1))['predictions']
    prediction = np.array(prediction)
    predicted_labels = prediction.argmax(axis=1)
    all_labels.extend(labels)
    all_predicted_labels.extend(predicted_labels)

Let's build the confusion matrix, to compare predicted labels with real labels for each class.

In [ ]:
import sklearn
import itertools
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(all_labels, all_predicted_labels)
plt.matshow(cm)
plt.title('Confusion matrix')
fmt = 'd'
thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], fmt),
            horizontalalignment="center",
            color="white" if cm[i, j] < thresh else "black")
plt.ylabel('True label')
plt.xlabel('Predicted label')
classes = range(10) # Labels are sorted 
tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes)
plt.yticks(tick_marks, classes)
plt.grid(False)
plt.show()

Let's check that we captured data (you may have to wait a minute or two for files to show up).

In [ ]:
%%sh -s "$s3_capture_path"

aws s3 ls --recursive $1

In [ ]:
%%sh -s "$s3_capture_path"

aws s3 cp --recursive $1 .

In [ ]:
!head tensorflow-training-200122-1035-014-666ddfaf-ep/AllTraffic/2020/01/22/13/51-29-573-13f40e87-d75d-487e-a5d5-e81ca695e097.jsonl

## Delete endpoint for best model

In [ ]:
import boto3
sm = boto3.client('sagemaker')
#sm.delete_endpoint(EndpointName=endpoint_name)

### https://gitlab.com/juliensimon/aim410
### Twitter: @julsimon